# Comparing Simple RAG vs Contextual RAG

This notebook compares the performance of our two RAG implementations:
1. Simple RAG: Basic document retrieval and question answering
2. Contextual RAG: Enhanced retrieval with document context and conversation history

We'll use sample documents and evaluate both systems on various question types.

In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Add project to path
sys.path.append('../')

# Load environment variables
load_dotenv()

## Initialize Both RAG Systems

In [ ]:
from simple_rag.modules.embedding import init_embeddings as simple_init_embeddings, init_llm as simple_init_llm
from simple_rag.modules.pdf_loader import PDFProcessor
from simple_rag.modules.qa_chain import QAChain as SimpleQAChain

from contextual_rag.modules.embedding import init_embeddings as contextual_init_embeddings, init_llm as contextual_init_llm
from contextual_rag.modules.pdf_loader import ContextualPDFProcessor
from contextual_rag.modules.qa_chain import ContextualQAChain

# PDF path - adjust this to your document(s)
pdf_path = "../data/mirage/sample_document.pdf"  # Update with your actual document path

# Initialize Simple RAG
simple_embeddings = simple_init_embeddings()
simple_llm = simple_init_llm()
simple_processor = PDFProcessor(simple_embeddings, persist_directory="../vector_db/simple")
if os.path.exists(pdf_path):
    simple_processor.load_and_process(pdf_path)
    simple_qa = SimpleQAChain(simple_processor.vector_store, simple_llm)
else:
    print(f"Warning: File {pdf_path} not found.")

# Initialize Contextual RAG
contextual_embeddings = contextual_init_embeddings()
contextual_llm = contextual_init_llm()
contextual_processor = ContextualPDFProcessor(contextual_embeddings, contextual_llm, persist_directory="../vector_db/contextual")
if os.path.exists(pdf_path):
    contextual_processor.load_and_process(pdf_path)
    contextual_qa = ContextualQAChain(contextual_processor.vector_store, contextual_llm)
else:
    print(f"Warning: File {pdf_path} not found.")

## Define Test Questions

We'll test various types of questions to compare performance.

In [ ]:
# Sample questions - update these based on your document content
test_questions = [
    "What is the main topic of the document?",
    "Can you summarize the key points?",
    "What are the conclusions presented?",
    # Add more questions specific to your document
]

# For contextual RAG conversation history test
conversation_questions = [
    "What is discussed in the introduction?",
    "What comes after that?",
    "Can you provide more details about it?",
    "What are the implications of this?"
    # The last three questions are deliberately vague to test contextual understanding
]

## Compare Simple and Contextual RAG on Basic Questions

In [ ]:
results = []

for question in test_questions:
    print(f"\nQuestion: {question}")
    
    # Simple RAG answer
    simple_answer = simple_qa.generate_answer(question)
    print(f"\nSimple RAG Answer:\n{simple_answer}")
    
    # Contextual RAG answer
    contextual_answer = contextual_qa.generate_answer(question)
    print(f"\nContextual RAG Answer:\n{contextual_answer}")
    
    # Store results
    results.append({
        "Question": question,
        "Simple RAG": simple_answer,
        "Contextual RAG": contextual_answer
    })

# Create DataFrame for comparison
results_df = pd.DataFrame(results)
results_df

## Test Contextual RAG with Conversation History

In [ ]:
# Test conversation with contextual understanding
conversation_history = []

for i, question in enumerate(conversation_questions):
    print(f"\nQ{i+1}: {question}")
    
    # Get answer with conversation history
    answer = contextual_qa.generate_answer(question, conversation_history)
    print(f"A{i+1}: {answer}")
    
    # Update conversation history
    conversation_history.append({"role": "user", "content": question})
    conversation_history.append({"role": "assistant", "content": answer})

## Evaluate Retrieval Quality

Let's compare the quality of document chunks retrieved by both systems.

In [ ]:
# Sample question for retrieval analysis
test_question = "What is the main topic of the document?"

# Get retrieved documents from Simple RAG
simple_docs = simple_qa.retriever.get_relevant_documents(test_question)
print(f"Simple RAG retrieved {len(simple_docs)} documents")

# Get retrieved documents from Contextual RAG
contextual_docs = contextual_qa._get_context(test_question)
print(f"Contextual RAG context length: {len(contextual_docs)}")

# Print first retrieved document from each system
print("\nSimple RAG first retrieved document:")
if simple_docs:
    print(simple_docs[0].page_content[:300], "...")

print("\nContextual RAG retrieved context (sample):")
print(contextual_docs[:300], "...")

## Performance Analysis

Based on our tests, we can analyze the performance of both systems:

### Advantages of Simple RAG
- Faster processing time
- Less computational overhead
- Simpler implementation
- Works well for straightforward, single-turn questions

### Advantages of Contextual RAG
- Better understanding of document context
- Improved handling of ambiguous queries
- Support for conversation history
- More coherent multi-turn interactions
- Better for complex document understanding

### Recommendations
- Use Simple RAG for: Basic QA, single-turn interactions, simpler documents
- Use Contextual RAG for: Complex documents, multi-turn conversations, ambiguous queries